# Testing Agentic approach to rubric evaluation

In [26]:
import openai
import glob, os, time, json, re, http, asyncio
from pydantic import BaseModel, Field
from enum import Enum
from agents import Agent, Runner, Tool, trace
from pydantic import BaseModel, Field
from enum import Enum
from typing import List

# Escape Codes para imprimir mensajes con color
Rd = "\033[1;31m"
Grn = "\033[1;32m"
Ylw = "\033[1;33m"
Blu = "\033[1;34m"
Mag = "\033[1;35m"
Rst = "\033[0m"

spacer = f"{80*'-'}"

In [67]:
def pretty_print(data, indent=0):
    """
    Recursively prints nested dictionaries, lists, and strings in a readable format.
    
    Args:
        data: The data structure to print (dict, list, str, etc.).
        indent: The current indentation level (used for nested structures).
    """
    # Define the indentation string
    spacing = "    " * indent

    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{Grn}{spacing}📌{key}{Rst}:")
            pretty_print(value, indent + 1)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print(f"{Mag}{spacing}➡️ [{index}]{Rst}")
            pretty_print(item, indent + 1)
    else:
        # Print strings or other primitive types
        print(f"{spacing}{data}")

Rubric Evaluation Agent

In [49]:
from typing import Union

class EvidenceStrength(str, Enum):
    EXPLICIT_STRONG = "Explicit & Strong Evidence"
    EXPLICIT_WEAK = "Explicit but Weak Evidence"
    IMPLICIT = "Implicit Evidence or Assumption"
    NONE = "No Evidence Provided"
    NOT_REQUIRED = "No Evidence Required (Trivial or Common Knowledge)"

class MissingEvidenceRubric(BaseModel):
    """Formato para evidencia faltante, que indica el campo específico y el motivo por el cual no se completó."""
    field: str = Field(description="Campo específico de la rúbrica que falta por completar.")
    motivo: str = Field(description="Motivo por el cual no se completó el campo. Puede ser 'No se encontró evidencia' o 'No aplica'.")


class Evidence(BaseModel):
    """Formato de evidencia, que indica si es claro y sustentado con evidencia."""
    judgement: str = Field(description=("Juicio sobre si se cumple con la rúbrica y por qué. Debe incluir una evaluación clara y fundamentada, indicando si las evidencias presentadas son suficientes, claras, y relevantes para cumplir con los criterios establecidos."))
    evidence: List[str] = Field(description=("Extracto exacto del texto que justifica el criterio evaluado. Debe ser un fragmento tal cual como viene en el texto provisto. Si hay varias evidencias, sepáralas por bullets o numeración."))
    clarity: str = Field(description="Indica si la evidencia es clara y coherente o si le falta claridad. Ej: 'Claridad adecuada' o 'Falta de explicación clara'.")
    referencia: EvidenceStrength = Field(description=(
            "Referencia o sustento científico de la evidencia. Puede ser cita bibliográfica, inferencia, "
            "conocimiento asumido, evidencia débil, o trivial. Las opciones posibles son:\n"
            "1. 'Explicit & Strong Evidence': Evidencia clara y robusta, con fuente confiable.\n"
            "2. 'Explicit but Weak Evidence': Fuente directa, pero débil o ambigua.\n"
            "3. 'Implicit Evidence or Assumption': Basado en conocimiento tácito o asumido, sin cita directa.\n"
            "4. 'No Evidence Provided': No se presenta evidencia respaldatoria.\n"
            "5. 'No Evidence Required (Trivial or Common Knowledge)': Conocimiento común, no requiere evidencia."
        ))
    manifestation: str = Field(description=(
    "Forma en que se manifiesta la evidencia dentro del documento evaluado. "
    "Las formas más comunes son:\n"
    "- 'Afirmación textual directa': Una afirmación explícita encontrada en el texto.\n"
    "- 'Hipervínculo': Enlace incluido en el texto que dirige a evidencia externa.\n"
    "- 'Cita bibliográfica': Referencia académica o técnica incluida como fuente.\n"
    "- 'Documento adjunto o anexo': Mención de un documento externo relacionado.\n"
    "- 'Otro': En caso de que la evidencia adopte una forma no convencional (especificar)."
))

class TechnicalMeritRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el mérito técnico del proyecto."""
    
    technical_evidence: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description=(
            "Evidencias del criterio relacionadas con la base técnica del proyecto. "
            "Debe incluir pruebas de la solidez técnica, enfoque metodológico, y coherencia de los fundamentos "
            "que sustentan el proyecto. Ejemplos incluyen modelos técnicos, análisis de viabilidad, "
            "y detalles sobre las tecnologías empleadas. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
        )
    )
    scientific_justification: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description=(
            "Evidencias relacionadas con la justificación científica que respalda el proyecto. "
            "Debe incluir la validación científica, teorías aplicadas, estudios previos relevantes, y el uso "
            "de referencias académicas que apoyan la propuesta. Ejemplos incluyen citas bibliográficas, "
            "resultados de investigaciones previas, o documentación que valida la base científica del proyecto. "
            "Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
        )
    )

class TechnicalSolutionRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la claridad y coherencia en la solución técnica propuesta."""
    
    solution_clarity: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias relacionadas con la claridad de la solución propuesta para abordar el problema. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    solution_coherence: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias relacionadas con la coherencia de la solución y su alineación con el problema. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    justification: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que sustentan la justificación de la solución técnica propuesta. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )

class TechnologyMaturityRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el nivel de maduración tecnológica según la escala TRL."""
    
    technology_maturity_level: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen el nivel de maduración tecnológica del proyecto según la escala TRL. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    evidence_of_progress: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias de los avances hacia el cumplimiento de TRL 4 y de los pasos realizados hacia un TRL superior. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    validation_evidence: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la validación realizada a la tecnología en las distintas etapas de desarrollo. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    plan_for_advancement: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias de los planes estructurados para avanzar a TRL 5 o superior, con pasos claros y definidos. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )

class ImplementationFeasibilityRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la viabilidad de la implementación considerando recursos y contexto."""
    
    implementation_feasibility: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen la viabilidad de la implementación en función de los recursos disponibles y el contexto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    resource_availability: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la disponibilidad de recursos necesarios para la implementación del proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    risk_assessment: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen los riesgos asociados con la implementación y las estrategias para mitigar estos riesgos. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    execution_strategy: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen la estrategia clara de ejecución del proyecto, con pasos bien definidos y cronograma adecuado. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    uncertainty_management: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran cómo se gestionan las incertidumbres y áreas de mejora dentro del proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )

class TeamCompositionRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la experiencia y trayectoria del equipo proponente en proyectos similares."""
    
    team_experience: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen la experiencia del equipo en proyectos similares y su capacidad técnica. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    relevant_background: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre el historial y antecedentes relevantes de los miembros del equipo en áreas clave. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    team_complementarity: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran la complementariedad de las habilidades dentro del equipo, asegurando una cobertura adecuada de todas las áreas necesarias para el proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    leadership_capabilities: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran las capacidades de liderazgo del equipo, incluyendo la gestión y la toma de decisiones. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    individual_expertise: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que destacan la experiencia individual de los miembros clave en el equipo, especialmente en áreas críticas para el proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )

class BudgetJustificationRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la distribución y justificación del presupuesto asignado al proyecto."""
    
    budget_clarity: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que la distribución del presupuesto está claramente definida y alineada con los objetivos del proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    resource_distribution: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la distribución de los recursos del presupuesto de forma coherente y equilibrada entre las diferentes necesidades del proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    justification_of_expenses: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran cómo se justifica cada gasto dentro del presupuesto, asegurando que cada partida tiene un propósito claro y relevante para el proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    optimization_opportunities: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que evalúan si el presupuesto ha sido optimizado adecuadamente o si existen áreas donde se podrían reducir gastos sin afectar el proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    budget_alignment: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que confirman que el presupuesto está alineado con los objetivos y la estrategia del proyecto, asegurando que los recursos se destinan donde son más necesarios. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    completeness: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que el presupuesto está completamente detallado, sin omisiones ni partidas faltantes. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )

class ExpectedImpactsMonitoringRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la definición de impactos esperados y su metodología de monitoreo."""
    
    impact_definition: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que los impactos esperados del proyecto están claramente definidos, detallados y alineados con los objetivos del proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    impact_relevance: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran que los impactos esperados están directamente relacionados con el problema a resolver, y cómo la solución propuesta contribuirá de manera significativa. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    monitoring_methodology: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la metodología de monitoreo propuesta para seguir el progreso de los impactos. Esto incluye métodos de seguimiento, indicadores y herramientas que se usarán. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    monitoring_plan: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias de que el plan de monitoreo es detallado y cubre las etapas necesarias del proyecto, incluyendo tiempos y formas de medición de los impactos. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    impact_assessment: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que indican cómo se evaluarán los impactos esperados, incluyendo cómo se medirá su éxito y qué métricas se utilizarán para su evaluación. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    adjustment_mechanism: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que describen los mecanismos de ajuste en caso de que los impactos no se logren según lo planeado, incluyendo acciones correctivas o alternativas. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )

class WorkPlanRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el plan de trabajo propuesto con hitos y cronograma."""
    
    work_plan_structure: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran que el plan de trabajo está bien estructurado, con una clara descripción de las tareas, los entregables y las fases del proyecto. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    milestones_definition: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que muestran que los hitos del proyecto están claramente definidos, con objetivos alcanzables y medibles para cada fase. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    timeline_and_schedule: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description=(
            "Evidencias que detallan el cronograma del proyecto, indicando los tiempos estimados para cada fase o tarea, "
            "y si son realistas y alcanzables. Nota: Si el cronograma proviene de un documento escaneado por OCR, "
            "es posible que esté distorsionado o incompleto. En caso de sospecha, levantar el comentario correspondiente. "
            "Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
        )
    )
    task_distribution: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran cómo se distribuyen las tareas entre los miembros del equipo y cómo se planifica el uso de recursos en función de los hitos. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    risk_and_contingency_planning: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la identificación de posibles riesgos y la existencia de un plan de contingencia para mitigarlos y asegurar el cumplimiento de los hitos. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    deliverable_quality: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que confirman que los entregables del proyecto están bien definidos, tienen criterios claros de calidad y son evaluables. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )

class IntellectualPropertyRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la propiedad intelectual en proceso o concedida."""
    
    ip_generation_potential: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que demuestran el potencial de generación de propiedad intelectual en el proyecto, como innovaciones tecnológicas, procesos, o productos originales. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    ip_protection_evidence: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias de que se ha realizado algún avance en la protección de la propiedad intelectual, como solicitudes de patentes, marcas, derechos de autor, u otros mecanismos de protección. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    ip_trademark_or_patent_status: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias del estado actual de las solicitudes de protección intelectual, como patentes o marcas registradas, y si están en trámite o si ya han sido concedidas. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    ip_management_strategy: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias sobre la existencia de una estrategia clara para la gestión y protección de la propiedad intelectual, como planificación, análisis de mercado, o acuerdos de licencia. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )
    ip_granted_or_advanced_status: List[Union[Evidence, MissingEvidenceRubric]] = Field(
        ...,
        description="Evidencias que indican que la propiedad intelectual ha sido concedida o está en proceso avanzado, mostrando la protección definida para los derechos relacionados. Si falta evidencia, se debe anotar con el formato adecuado de MissingEvidenceRubric."
    )


class MaturityRubric(BaseModel):
    """Formato de Rúbrica, para juntar las evidencias relacionadas a la madurez de un proyecto científico."""
    # Criterio de mérito técnico
    merito_tecnico: TechnicalMeritRubicCriterion = Field(
        description="Evidencias relacionadas a la base técnica del proyecto y su justificación científica."
    )
    
    # Criterio de solución técnica
    solucion_tecnica: TechnicalSolutionRubicCriterion = Field(
        description="Evidencias relacionadas a la claridad y coherencia de la solución técnica propuesta para abordar el problema."
    )
    
    # Criterio de madurez tecnológica
    madurez_tecnologica: TechnologyMaturityRubicCriterion = Field(
        description="Evidencias relacionadas al nivel de maduración tecnológica del proyecto según la escala TRL."
    )
    
    # Criterio de viabilidad de implementación
    viabilidad_implementacion: ImplementationFeasibilityRubicCriterion = Field(
        description="Evidencias relacionadas a la viabilidad de implementación considerando recursos y contexto."
    )
    
    # Criterio de composición del equipo
    equipo_proponente: TeamCompositionRubicCriterion = Field(
        description="Evidencias relacionadas a la experiencia y trayectoria del equipo en proyectos similares."
    )
    
    # Criterio de justificación del presupuesto
    justificacion_presupuesto: BudgetJustificationRubicCriterion = Field(
        description="Evidencias relacionadas a la distribución y justificación del presupuesto asignado al proyecto."
    )
    
    # Criterio de impactos esperados y su metodología de monitoreo
    impactos_monitoreo: ExpectedImpactsMonitoringRubicCriterion = Field(
        description="Evidencias relacionadas a la definición clara de los impactos esperados y su monitoreo."
    )
    
    # Criterio de plan de trabajo propuesto con hitos y calendario
    plan_trabajo: WorkPlanRubicCriterion = Field(
        description="Evidencias relacionadas a la claridad del plan de trabajo con entregables y cronograma."
    )
    
    # Criterio de propiedad intelectual en proceso o concedida
    propiedad_intelectual: IntellectualPropertyRubicCriterion = Field(
        description="Evidencias relacionadas a la existencia o potencial de generación de propiedad intelectual."
    )

# Rubric Evaluation

In [50]:
rubric_evaluator_agent = Agent(
    name="Rubric Evaluator Agent",
    instructions="""
    Description:
    The Rubric Evaluator Agent conducts a rigorous initial evaluation of a project proposal based on a structured maturity assessment rubric. Its core function is to extract, assess, and organize **hard evidence** from the proposal, aligning it to each rubric criterion. This agent prioritizes **thoroughness and honesty**: it does **not assume or infer evidence** and is **not obligated to complete all rubric fields** if no solid evidence exists.

    Approach:
    - **Extensively search for explicit, traceable evidences** related to each criterion in the maturity rubric. It is possible that you might find more than one evidence for each criterion. Might be possible you find no evidence at all.
    - **Avoid speculative judgments**: if there is no clear or strong evidence, the agent must **leave the field unscored or flag it accordingly**.
    - If evidence is strong, the agent should **confidently acknowledge it**. If evidence is weak, limited, or ambiguous, this should also be made clear.
    - **If the evidence is NOT found, explicitly flag it as missing.**

    Inputs:
    1. A project or proposal document containing technical, methodological, organizational, and contextual information.
    2. A predefined maturity rubric with clearly described criteria (e.g., TRL, technical merit, feasibility, impacts, team experience).

    Output:
    A structured object (`MaturityRubric`) where each field corresponds to a rubric criterion and contains an evidence-grounded analysis. The output:
    - Highlights **whether the criterion is strongly supported, weakly supported, or unsupported**.
    - May leave fields empty if **no verifiable evidence is found**.

    Guidelines:
    - The agent **does not evaluate based on potential**—only on present, explicit information.
    - The output should include direct references to the text or reasoning showing **why** a criterion is (or isn't) fulfilled.
    - Do **not be afraid to say the evidence is insufficient**, vague, or absent.
    - **Flag missing evidence explicitly.**
    
    Considerations:
    - The information is provided from an OCR scan of a PDF document. Don't hesitate to highlight that maybe some evidences are there but there might be importation issues. Clasically, this is the case with tables or images (chronograms for instance).
    
    Workflow Position:
    This agent works **prior to** the deliberation phase with other agents (e.g., Proponent, Devil’s Advocate). It establishes a **fact-based baseline**, grounded in evidence, that serves to inform and constrain further discussion.

    Adjustment:
    The agent can revise its output if new documentation is submitted or clarification is provided later in the evaluation process.
    """,
    tools=[],
    output_type=MaturityRubric,
    model="gpt-4o-mini-2024-07-18",
)



In [51]:
with open('./QRO/779_Aguadelluvia.txt', 'rb') as file:
    file_content = file.read()
    file_content = file_content.decode('utf-8', errors='ignore')

In [52]:
with trace("Prueba de Extracción de Evidencias"):
    result = await Runner.run(
        rubric_evaluator_agent,
        input=f"Cumple con tu rol de trabajo, responde en español. El contenido viene a continuación: \n{file_content}",
        )


In [68]:
pretty_print(result.final_output.model_dump())

📌merito_tecnico:
    📌technical_evidence:
        ➡️ [0]
            📌judgement:
                La propuesta menciona avances en la investigación sobre reactores de oxidación electroquímica, los cuales se han evaluado y están respaldados por publicaciones científicas y patentes. La referencia a estudios previos y resultados de investigaciones añade credibilidad a la base técnica del proyecto.
            📌evidence:
                ➡️ [0]
                    El proyecto se basa en la tecnología de oxidación avanzada para el tratamiento de agua, la cual ha sido validada en varias publicaciones científicas.
            📌clarity:
                Claridad adecuada
            📌referencia:
                EvidenceStrength.EXPLICIT_STRONG
            📌manifestation:
                Afirmación textual directa
    📌scientific_justification:
        ➡️ [0]
            📌judgement:
                Se presentan múltiples referencias a estudios previos, aportando una justificación científica sólida

In [31]:
for rubric, evidences in result.final_output.model_dump().items():
    print(f"➤ {Ylw}{rubric}{Rst}")
    for evidence_type, evidence_list in evidences.items():
        print(f"\t➕{Grn}{evidence_type}{Rst}:")
        for i, evidence in enumerate(evidence_list, start=1):
            print(f"\t  {i}. ✅{evidence['judgement']}")
            print(f"\t     📋{evidence['evidence']}")
            print(f"\t     {Mag}Evidence Manifestation:{Rst} {evidence['manifestation']}")
            print(f"\t     {Mag}Clarity:{Rst} {evidence['clarity']} {Mag}Fortaleza de Evidencia:{Rst} {evidence['referencia'].value}\n")
    print(spacer)

➤ merito_tecnico
	➕technical_evidence:
	  1. ✅La propuesta presenta un análisis sólido sobre la tecnología electroquímica, respaldado por el desarrollo de reactores que han mostrado resultados positivos tanto en laboratorio como en condiciones reales, lo que indica una robustez técnica.
	     📋['"la invención tecnológica funciona adecuadamente tanto a nivel laboratorio, como en condiciones reales de operación"', '"el concepto se ha validado"']
	     Evidence Manifestation: Afirmación textual directa
	     Clarity: Adecuada. Fortaleza de Evidencia: Explicit & Strong Evidence

	  2. ✅Los modelos de predicción de precipitación se fundamentan en investigaciones previas, asegurando que el enfoque propuesto tenga un respaldo científico.
	     📋['"la integración de modelos meteorológicos globales y regionales es un enfoque fundamental para mejorar la calidad y resolución de los pronósticos"']
	     Evidence Manifestation: Afirmación textual directa
	     Clarity: Adecuada. Fortaleza de Eviden

# Evaluation Stage

In [32]:
from agents import function_tool

@function_tool
def original_document() -> str:
    """Provides the original document, that desires funding from our institution."""
    return file_content

@function_tool
def rubric_facts() -> MaturityRubric:
    """Provides the list of found facts that align to the rubric. Contains evidence extracted from the original document. Done previously by an evaluator."""
    return result.final_output

@function_tool
def rubric_format() -> str:
    """Thie is the rubric for evluation."""
    text = """
    # Rúbrica de Evaluación de Proyectos de Investigación

    ## 1. Mérito técnico
    _Evaluación de la base científica y técnica del proyecto._

    - **5 puntos**: Justificación técnica excelente, con respaldo sólido y claridad.
    - **4 puntos**: Base técnica bien sustentada.
    - **3 puntos**: Justificación aceptable, aunque con áreas de mejora.
    - **2 puntos**: Fundamento débil o poco claro.
    - **1 punto**: Justificación técnica vaga o sin referencias claras.
    - **0 puntos**: Sin justificación técnica.

    ---

    ## 2. Solución técnica propuesta
    _Claridad, coherencia e innovación de la solución planteada al problema._

    - **5 puntos**: Solución innovadora, bien formulada y alineada al problema.
    - **4 puntos**: Solución clara y estructurada, con justificación sólida.
    - **3 puntos**: Solución adecuada pero con áreas poco desarrolladas.
    - **2 puntos**: Solución identificable, pero con poca claridad o detalle.
    - **1 punto**: Planteamiento vago o no relacionado con el problema.
    - **0 puntos**: Sin una solución clara o coherente.

    ---

    ## 3. Nivel de madurez tecnológica (TRL ≥ 4 preferente)
    _Estado de desarrollo tecnológico del proyecto._

    - **5 puntos**: TRL 4 con evidencia sólida y plan claro hacia TRL 5 o más.
    - **4 puntos**: TRL 4 validado en laboratorio.
    - **3 puntos**: TRL 4 con validación parcial.
    - **2 puntos**: TRL 3 con intención de avanzar a TRL 4, validación limitada.
    - **1 punto**: TRL 3 sin evidencia de avance.
    - **0 puntos**: TRL menor a 3 o sin evidencia.

    ---

    ## 4. Viabilidad de implementación
    _Factibilidad técnica y operativa con los recursos disponibles._

    - **5 puntos**: Proyecto claramente viable, con estrategia de ejecución.
    - **4 puntos**: Implementación bien fundamentada, con riesgos controlables.
    - **3 puntos**: Viabilidad aceptable, pero con incertidumbres.
    - **2 puntos**: Posible, pero con riesgos altos sin mitigación clara.
    - **1 punto**: Dificultades significativas para implementarse.
    - **0 puntos**: No viable en las condiciones actuales.

    ---

    ## 5. Equipo proponente (perfil y trayectoria)
    _Experiencia del equipo en proyectos similares._

    - **5 puntos**: Equipo altamente calificado, con trayectoria sólida y liderazgo.
    - **4 puntos**: Equipo con experiencia y capacidades complementarias.
    - **3 puntos**: Equipo adecuado, con áreas de mejora en trayectoria.
    - **2 puntos**: Algún miembro con experiencia, pero equipo débil.
    - **1 punto**: Experiencia limitada o poco clara.
    - **0 puntos**: Sin experiencia ni evidencia de capacidad técnica.

    ---

    ## 6. Justificación del presupuesto
    _Coherencia entre los recursos solicitados y los objetivos._

    - **5 puntos**: Presupuesto detallado, óptimo y bien justificado.
    - **4 puntos**: Presupuesto estructurado y alineado a los objetivos.
    - **3 puntos**: Presupuesto aceptable, con oportunidades de optimización.
    - **2 puntos**: Justificación parcial, con áreas no justificadas.
    - **1 punto**: Distribución poco clara o inconsistente.
    - **0 puntos**: Presupuesto sin justificación o con errores graves.

    ---

    ## 7. Impactos esperados y monitoreo
    _Definición clara de impactos y su seguimiento._

    - **5 puntos**: Impactos sólidos con monitoreo definido y estructurado.
    - **4 puntos**: Impactos bien definidos con monitoreo adecuado.
    - **3 puntos**: Impactos razonables, con metodología mejorable.
    - **2 puntos**: Impactos definidos, pero monitoreo débil.
    - **1 punto**: Impactos mencionados sin relación clara con el problema.
    - **0 puntos**: No se definen impactos ni forma de medirlos.

    ---

    ## 8. Plan de trabajo (hitos y calendario)
    _Claridad del cronograma y entregables._

    - **5 puntos**: Plan detallado, con hitos definidos y calendario realista.
    - **4 puntos**: Plan estructurado, con hitos claros y alcanzables.
    - **3 puntos**: Hitos definidos, pero calendario poco claro.
    - **2 puntos**: Plan poco estructurado o incompleto.
    - **1 punto**: Plan desorganizado o sin entregables definidos.
    - **0 puntos**: No presenta plan de trabajo.

    ---

    ## 9. Propiedad intelectual (preferente)
    _Potencial o existencia de generación de propiedad intelectual._

    - **5 puntos**: Propiedad intelectual concedida o proceso avanzado y claro.
    - **4 puntos**: En trámite, con estrategia clara.
    - **3 puntos**: Avances hacia protección identificados.
    - **2 puntos**: Posible generación, pero sin evidencia clara.
    - **1 punto**: Bajo potencial de generación.
    - **0 puntos**: No se considera o no hay intención.

    """
    return text

In [45]:
from typing import Union

class MissingRubric(BaseModel):
    field: str = Field(description="Campo específico de la rúbrica que falta por completar.")
    motivo: str = Field(description="Motivo por el cual no se completó el campo. Puede ser 'No se encontró evidencia' o 'No aplica'.")

class MaturityRubric(BaseModel):
    """Formato de Rúbrica, para juntar las evidencias relacionadas a la madurez de un proyecto científico."""
    # Criterio de mérito técnico
    merito_tecnico: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la base técnica del proyecto y su justificación científica. Comentarios sobre la solidez técnica y científica. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    solucion_tecnica: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la claridad y coherencia de la solución técnica propuesta para abordar el problema. Comentarios sobre la solución técnica. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    madurez_tecnologica: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas al nivel de maduración tecnológica del proyecto según la escala TRL. Comentarios sobre el nivel de madurez tecnológica. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    viabilidad_implementacion: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la viabilidad de implementación considerando recursos y contexto. Comentarios sobre la factibilidad de implementación. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    equipo_proponente: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la experiencia y trayectoria del equipo en proyectos similares. Comentarios sobre la composición del equipo. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    justificacion_presupuesto: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la distribución y justificación del presupuesto asignado al proyecto. Comentarios sobre la justificación del presupuesto. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    impactos_monitoreo: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la definición clara de los impactos esperados y su monitoreo. Comentarios sobre los impactos esperados y su monitoreo. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    plan_trabajo: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la claridad del plan de trabajo con entregables y cronograma. Comentarios sobre el plan de trabajo. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )
    
    propiedad_intelectual: List[Union[str, MissingRubric]] = Field(
        description="Se dirige a evidencias relacionadas a la existencia o potencial de generación de propiedad intelectual. Comentarios sobre la propiedad intelectual. Puede ser una lista de textos o una lista de objetos MissingRubric si no se encuentra evidencia."
    )

class Evaluation(BaseModel):
    pitch: str = Field(description="El pitch del proyecto, que se desea evaluar.")
    rubric: MaturityRubric = Field(description="Opinión sobre la rúbrica de evaluación, que contiene los criterios y evidencias encontradas. ¿Está completo o falta?")

class Observation(BaseModel):
    """Observación sobre la evaluación."""
    original: str = Field(description="Extracto de la observación original, que servirá como contexto para tu análisis. Asegúrate de incluir suficiente información para que sea comprensible.")
    observation: str = Field(description="Escribe tu observación, apoyo, crítica o refutación sobre la retroalimentación conferida. Sé claro, conciso y fundamenta tu respuesta.")

class Observation_to_critique(BaseModel):
    """Observación sobre la evaluación de otros."""
    reviewer: str = Field(description="Escribe el nombre del reviewer quien estás criticando o apoyando.")
    original: str = Field(description="Extracto de la observación original, que servirá como contexto para tu análisis. Asegúrate de incluir suficiente información para que sea comprensible.")
    observation: str = Field(description="Escribe tu observación, apoyo, crítica o refutación sobre la retroalimentación conferida. Sé claro, conciso y fundamenta tu respuesta.")

class Observations(BaseModel):
    """Observaciones sobre la evaluación."""
    observations: List[Observation] = Field(description="Escribe la observación, apoyo crítica o refutación sobre la retroalimentación conferida de acuerdo a tu rol.")

class Observations_critiques(BaseModel):
    """Observaciones sobre la evaluación."""
    observations: List[Observation_to_critique] = Field(description="Escribe la observación, apoyo crítica o refutación sobre la retroalimentación conferida de acuerdo a tu rol.")
    
class Project_Manager_Evaluation(BaseModel):
    """Evaluación del Project Manager."""
    plan: List[Observation] = Field(
        description=(
            "Observaciones relacionadas al plan del proyecto. Evalúa si el plan está bien estructurado, "
            "con hitos claros, cronograma realista y entregables definidos. Identifica posibles áreas de mejora "
            "en la organización y planificación del proyecto."
        )
    )
    recurso_tiempo: List[Observation] = Field(
        description=(
            "Observaciones sobre la gestión de tiempo del proyecto. Analiza si los tiempos asignados a cada tarea "
            "son realistas y alcanzables. Identifica posibles riesgos de retrasos y su impacto en el cronograma general."
        )
    )
    recurso_dinero: List[Observation] = Field(
        description=(
            "Observaciones sobre la gestión de dinero del proyecto. Evalúa si el presupuesto está bien distribuido "
            "y justificado. Asegúrate de que los recursos financieros se asignen a las áreas críticas del proyecto, "
            "considerando posibles omisiones o gastos innecesarios. Verifica si los salarios cumplen con estándares "
            "mínimos (e.g., salario mínimo mexicano) y si hay áreas donde el presupuesto podría optimizarse."
        )
    )
    viability: List[Observation] = Field(
        description=(
            "Observaciones sobre la viabilidad del proyecto desde múltiples ángulos. Evalúa si el proyecto es "
            "realizable considerando los recursos disponibles, el contexto operativo, y los riesgos asociados. "
            "Incluye análisis sobre la claridad de los objetivos, la alineación con las capacidades del equipo, "
            "la disponibilidad de infraestructura, y la factibilidad técnica. Identifica posibles problemas como "
            "falta de recursos, riesgos no mitigados, dependencias externas críticas, o cronogramas poco realistas. "
            "Si el proyecto está bien estipulado y creíble, se debe destacar su solidez y viabilidad."
        )
    )

class Business_Designer_Evalaution(BaseModel):
    """Evaluación del Business Designer."""	
    commercial_viability: List[Observation] = Field(
        description=(
            "Observaciones sobre la viabilidad comercial del proyecto. Evalúa si el proyecto tiene un mercado "
            "definido, una propuesta de valor clara y un modelo de negocio sostenible. Identifica posibles riesgos "
            "comerciales y su impacto en la viabilidad del proyecto."
        )
    )
    market_necessity: List[Observation] = Field(
        description=(
            "Observaciones sobre la necesidad del mercado para el proyecto. Evalúa si el proyecto aborda un problema "
            "real y relevante en el mercado. Identifica posibles áreas de mejora en la identificación de necesidades "
            "del cliente y su alineación con la solución propuesta."
        )
    )
    market_timing: List[Observation] = Field(
        description=(
            "Observaciones sobre el momento del mercado para el proyecto. Evalúa si el mercado está saturado con demasiada "
            "competencia y bajos rendimientos, o si es un mercado novedoso con alto valor agregado. Incluye comentarios sobre "
            "si el mercado está consolidado con grandes jugadores o si aún no lo está."
        )
    )
    impact: List[Observation] = Field(
        description=(
            "Observaciones sobre el impacto del proyecto. Evalúa si el proyecto tiene un impacto positivo en la "
            "sociedad, el medio ambiente o la economía. Identifica posibles áreas de mejora en la medición y "
            "evaluación del impacto del proyecto."
        )
    )
    
class Consolidated_Evaluation(BaseModel):
    """Evaluación consolidada."""
    consolidated_project_pitch: Evaluation = Field(description="Evaluación del proyecto. Obteido leyendo a partir de documento y agregando las observaciones de los evaluadores.")
    fortalezas_de_propuesta: List[str] = Field(description="A partir del documento y de la discusión, se obienen las Fortalezas de la propuesta. ¿Cuáles son los puntos fuertes del planteamiento proyecto?")
    debilidades_de_propuesta: List[str] = Field(description="A partir del documento y de la discusión, se obienen las Debilidades de la propuesta. ¿Cuáles son los puntos débiles del planteamiento proyecto?")
    recomendaciones: List[str] = Field(description="A partir del documento y de la discusión, se obienen las Recomendaciones. ¿Cuáles son las recomendaciones para mejorar el planteamiento proyecto?")
        

In [46]:
visionary_domain_specialist = Agent(
    name="Visionary Domain Specialist",
    instructions="""
    instructions:
    The Visionary Domain Specialist Agent is tasked with identifying the specific domain or field of expertise required for the proposal by analyzing the original document, rubric format, and rubric facts. This agent acts as a domain specialist, adapting its focus based on the content and context of the document. As a visionary evaluator for the company, the agent's objective is to create value by identifying and supporting projects with high potential for success and innovation.

    Inputs:
    1. The original document containing the proposal, background, and context.
    2. The rubric format outlining the evaluation criteria.
    3. The rubric facts providing additional context or guidelines for evaluation.

    Objective:
    1. Analyze the document to determine the relevant domain or field of expertise (e.g., technology, healthcare, education, etc.).
    2. Provide insights and recommendations specific to the identified domain, ensuring that the proposal aligns with best practices, trends, and innovations in that field.
    3. Highlight domain-specific strengths, opportunities, and potential for value creation within the proposal.
    4. Identify weaknesses or areas for improvement, offering constructive feedback to enhance the proposal's quality and alignment with the rubric criteria.
    5. Advocate for the proposal by emphasizing its visionary aspects and potential to drive innovation and success.

    Adjust to Feedback:
    1. If additional context or clarification is provided, refine the domain analysis and recommendations accordingly.
    2. Adapt the critique and insights based on new information or adjustments to the proposal.
    3. Be prepared to defend the generated feedback if questioned, providing justification and evidence for the positions taken.

    Output:
    The agent's output will include:
    - Strengths: Key aspects of the proposal that align with the rubric and demonstrate high potential for success.
    - Weaknesses: Areas where the proposal could be improved, with specific suggestions for enhancement.

    Workflow Position:
    This agent operates early in the evaluation process, setting the stage for domain-specific considerations that other agents may build upon. The agent's visionary perspective ensures that promising projects are given the attention and support needed to maximize their potential. The feedback provided will be cross-examined later in the evaluation process.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Evaluation,
)

realist_domain_specialist = Agent(
    name="Realist Domain Specialist",
    instructions="""
    instructions:
    The Realist Domain Specialist Agent operates as a devil's advocate within the specific domain or field identified in the proposal. This agent's primary role is to critically evaluate the proposal by questioning its value and feasibility, considering the stakes involved if the project is approved. The agent grounds its analysis in the current state of the field, identifying how things are done today and using this as a baseline to challenge the proposal's assumptions, innovations, and potential impact.

    Inputs:
    1. The original document containing the proposal, background, and context.
    2. The rubric format outlining the evaluation criteria.
    3. The rubric facts providing additional context or guidelines for evaluation.

    Objective:
    1. Identify the current state of the domain or field as described in the document, including standard practices, methodologies, and benchmarks.
    2. Critically assess the proposal's claims, questioning its value, feasibility, and alignment with the current state of the field.
    3. Highlight potential risks, weaknesses, or oversights in the proposal, particularly where it diverges from established practices or fails to address key challenges.
    4. Provide constructive feedback to refine the proposal, ensuring it addresses the stakes and challenges inherent in its implementation.

    Adjust to Feedback:
    1. If additional context or clarification is provided, refine the critique and recommendations accordingly.
    2. Adapt the analysis based on new information or adjustments to the proposal.
    3. Be prepared to defend the critique if questioned, providing justification and evidence for the positions taken.

    Output:
    The agent's output will include:
    - A detailed analysis of the current state of the field, serving as a baseline for evaluation.
    - A critique of the proposal's value and feasibility, with specific references to risks, weaknesses, and areas for improvement.
    - Constructive feedback aimed at enhancing the proposal's alignment with the domain's realities and addressing potential stakes.

    Workflow Position:
    This agent operates early in the evaluation process, providing a critical perspective that challenges the proposal's assumptions and ensures that its value and feasibility are rigorously tested. The feedback provided will be cross-examined later in the evaluation process.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Evaluation,
)

project_manager_agent = Agent(
    name="Project Manager Agent",
    instructions="""
    instructions:
    The Project Manager Agent evaluates the proposal exclusively from a project management perspective. This agent focuses on assessing timelines, budgets, resource allocation, and overall project execution capabilities. It does not delve into technical or domain-specific aspects but ensures that the project is feasible and well-structured from a management standpoint.

    Inputs:
    1. The proposal document, including timelines, budgets, and resource plans.
    2. Rubric outlining project management criteria (e.g., milestones, deliverables, risk management, resource allocation).

    Objective:
    1. Evaluate the clarity and feasibility of the proposed timeline, ensuring that milestones and deliverables are realistic and achievable.
    2. Assess the budget, verifying whether it aligns with the scope of the project and whether resource allocation is sufficient and appropriate.
    3. Identify potential risks related to project execution, such as resource shortages, unrealistic deadlines, or insufficient contingency planning.
    4. Provide recommendations to improve project management aspects, such as refining timelines, reallocating resources, or enhancing risk mitigation strategies.

    Adjust to Feedback:
    1. If additional details or clarifications are provided, refine the evaluation to reflect the updated information.
    2. Adapt recommendations based on new insights or adjustments to the project plan.

    Output:
    The agent's output will include:
    - A detailed assessment of the project timeline, budget, and resource allocation.
    - Identification of risks and areas for improvement in project management.
    - Constructive feedback to enhance the feasibility and structure of the project from a management perspective.

    Workflow Position:
    This agent operates early in the evaluation process, ensuring that the project is well-grounded in sound project management principles. Its feedback serves as a foundation for assessing the project's overall viability and execution potential.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Project_Manager_Evaluation,
)

business_designer_agent = Agent(
    name="Business Designer Agent",
    instructions="""
    Description:
    The Business Designer Agent evaluates the proposal from a business design perspective, focusing on the value proposition, business model, market fit, and scalability. This agent ensures that the proposal aligns with sound business principles and has the potential to create sustainable value.

    Inputs:
    1. The proposal document, including details on the value proposition, target market, revenue model, and scalability plans.
    2. Rubric outlining business design criteria (e.g., market analysis, competitive advantage, customer segmentation, revenue streams).

    Objective:
    1. Assess the clarity and strength of the value proposition, ensuring it addresses a real market need or problem.
    2. Evaluate the business model, including revenue streams, cost structure, and scalability potential.
    3. Analyze the target market and customer segmentation, identifying whether the proposal demonstrates a strong understanding of its audience.
    4. Identify potential risks or weaknesses in the business design, such as unclear revenue models, limited market potential, or scalability challenges.
    5. Provide constructive feedback to refine the business aspects of the proposal, ensuring alignment with best practices and market trends.

    Adjust to Feedback:
    1. If additional details or clarifications are provided, refine the evaluation to reflect the updated information.
    2. Adapt recommendations based on new insights or adjustments to the business model or market strategy.

    Output:
    The agent's output will include:
    - A detailed assessment of the value proposition, business model, and market fit.
    - Identification of risks and areas for improvement in the business design.
    - Constructive feedback to enhance the proposal's business viability and scalability.

    Workflow Position:
    This agent operates early in the evaluation process, ensuring that the proposal is grounded in a strong business foundation. Its feedback serves as a basis for assessing the proposal's potential for long-term success and sustainability.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Project_Manager_Evaluation,
)

orchestator_agent = Agent(
    name="Orchestator Agent",
    instructions="""
    Description:
    The Orchestrator Agent consolidates the knowledge, observations, and concerns provided by all previous agents to produce a comprehensive and balanced evaluation of the proposal. Its primary goal is to synthesize the value proposition, assess its adherence to the rubric, and provide a final list of strengths and weaknesses. This agent ensures that the company invests in projects with strong foundations and high potential for success.

    Approach:
    - Gather and analyze inputs from all agents, including their observations, critiques, and recommendations.
    - Synthesize a clear and concise value proposition based on the consolidated feedback.
    - Evaluate how well the proposal adheres to the rubric, highlighting areas of alignment and misalignment.
    - Identify and prioritize strengths and weaknesses, discarding opinions that are overly controversial, convoluted, or unsupported by evidence.
    - Maintain a critical yet constructive perspective, ensuring that the final evaluation supports sound decision-making.

    Inputs:
    1. Consolidated outputs from all agents, including their observations, critiques, and recommendations.
    2. The original proposal document and rubric for reference.

    Objective:
    1. Write down the consolidated value proposition, ensuring it reflects the core strengths and potential of the proposal.
    2. Assess the proposal's adherence to the rubric, providing a balanced evaluation of its alignment with the criteria.
    3. Enlist the final list of strengths and weaknesses, prioritizing clarity, relevance, and actionable insights.
    4. Discard overly controversial or convoluted opinions that do not contribute to a constructive evaluation.
    5. Ensure the final output supports the company's goal of investing in projects with strong foundations and high potential for success.

    Output:
    The agent's output will include:
    - A consolidated value proposition.
    - An evaluation of the proposal's adherence to the rubric.
    - A prioritized list of strengths and weaknesses, with actionable insights.

    Workflow Position:
    This agent operates at the final stage of the evaluation process, synthesizing all prior inputs to provide a comprehensive and balanced assessment. Its output serves as the basis for the company's decision-making, ensuring that only well-founded and high-potential projects are pursued.
    """,
    tools=[original_document, rubric_format, rubric_facts],
    model="gpt-4o-2024-08-06",
    output_type=Consolidated_Evaluation,
)


## Step 1: Evaluation

In [47]:
import asyncio

msg = ("Tienes acceso a documentos clave: la rúbrica de evaluación, las evidencias extraídas de la rúbrica, y el documento original de la propuesta. "
    "Tu tarea es analizar la propuesta con un enfoque equilibrado: identificar proyectos con alto potencial de innovación que puedan generar valor para la empresa, "
    "pero también ser crítico y objetivo, ya que estamos invirtiendo nuestros propios recursos. "
    "Evalúa cuidadosamente los riesgos y oportunidades, asegurándote de que las decisiones estén fundamentadas en evidencia sólida y alineadas con los criterios de la rúbrica.")

with trace("MAAF Evaluation"):
    s1_DomSpecVisionary, s1_DomSpecRealist, s1_ProjManager, s1_BizDesigner = await asyncio.gather(
        Runner.run(
            visionary_domain_specialist,
            input=f"{msg}.",
            ),
        Runner.run(
            realist_domain_specialist,
            input=f"{msg}.",
            ),
        Runner.run(
            project_manager_agent,
            input=f"{msg}.",
            ),
        Runner.run(
            business_designer_agent,
            input=f"{msg}.",
            ),
    
    )


In [48]:
print(spacer)
print(f"{Ylw}Visionary Domain Specialist{Rst}")
pretty_print(s1_DomSpecVisionary.final_output.model_dump())

print(spacer)
print(f"{Ylw}Realist Domain Specialist{Rst}")
pretty_print(s1_DomSpecRealist.final_output.model_dump())

print(spacer)
print(f"{Ylw}Project Manager{Rst}")
pretty_print(s1_ProjManager.final_output.model_dump())

print(spacer)
print(f"{Ylw}Business Designer{Rst}")
pretty_print(s1_BizDesigner.final_output.model_dump())

--------------------------------------------------------------------------------
Visionary Domain Specialist
pitch:
    El proyecto aborda el desarrollo de técnicas avanzadas para la captación y tratamiento del agua de lluvia mediante herramientas innovadoras que combinan modelos predictivos y reactores electroquímicos en Querétaro. Se enfoca en soluciones tecnológicas que potencian la economía circular, asegurando un uso sostenible del recurso hídrico en la región.
rubric:
    merito_tecnico:
        - [0]
            La propuesta presenta un análisis sólido sobre la tecnología electroquímica, respaldado por el desarrollo de reactores que han mostrado resultados positivos tanto en laboratorio como en condiciones reales, lo que indica una robustez técnica.
        - [1]
            Los modelos de predicción de precipitación se fundamentan en investigaciones previas, asegurando que el enfoque propuesto tenga un respaldo científico.
        - [2]
            Los artículos citados validan

In [104]:
from IPython.display import Markdown
Markdown(f"""
# Pitch
{result.final_output.pitch}

- **Mérito Técnico:** {result.final_output.rubric.merito_tecnico}

- **Solución Técnica:** {result.final_output.rubric.solucion_tecnica}

- **Madurez Tecnológica:** {result.final_output.rubric.madurez_tecnologica}

- **Viabilidad de Implementación:** {result.final_output.rubric.viabilidad_implementacion}

- **Equipo Proponente:** {result.final_output.rubric.equipo_proponente}

- **Justificación del Presupuesto:** {result.final_output.rubric.justificacion_presupuesto}

- **Impactos y Monitoreo:** {result.final_output.rubric.impactos_monitoreo}

- **Plan de Trabajo:** {result.final_output.rubric.plan_trabajo}

- **Propiedad Intelectual:** {result.final_output.rubric.propiedad_intelectual}
""")


# Pitch
La propuesta plantea la creación de una tecnología integrada en Querétaro para el uso eficiente del agua de lluvia mediante sistemas predictivos y reactores electroquímicos de oxidación avanzada. Esta solución se enfoca en la economía circular, permitiendo el tratamiento y reuso del agua de lluvia y gris, con un enfoque ambientalmente sustentable y socialmente rentable, adaptándose a áreas urbanas y rurales sin acceso a electricidad convencional.

- **Mérito Técnico:** La propuesta tiene un mérito técnico elevado con una justificación científica y técnica bien definida. Está respaldada por publicaciones, patentes y una clara trayectoria académica del equipo, lo que fortalece su fundamento técnico.

- **Solución Técnica:** La solución técnica es clara y coherente, combinando tecnología predictiva con tratamiento innovador de agua, lo que plantea una respuesta efectiva al problema descrito.

- **Madurez Tecnológica:** Esta propuesta ha alcanzado un TRL 4, y existe un plan bien desarrollado para avanzar al TRL 5, demostrando un sólido camino de desarrollo tecnológico.

- **Viabilidad de Implementación:** El proyecto es claramente viable, con un enfoque modular adaptable a diferentes contextos y apoyado por un profundo análisis de riesgos y factibilidad.

- **Equipo Proponente:** El equipo proponente tiene una fuerte trayectoria y experiencia, lo que asegura una capacidad técnica y operacional para ejecutar el proyecto con éxito.

- **Justificación del Presupuesto:** El presupuesto está bien estructurado y justificado, alineado con los objetivos del proyecto, aunque podría optimizarse aún más en algunos puntos específicos.

- **Impactos y Monitoreo:** Los impactos esperados están bien definidos con un sistema de monitoreo comprensivo, asegurando evaluaciones tanto ambientales como sociales.

- **Plan de Trabajo:** El plan de trabajo es detallado y realista, con hitos claramente definidos y entregables en un cronograma factible.

- **Propiedad Intelectual:** El proyecto incluye esfuerzos significativos para la protección de propiedad intelectual con varias patentes en trámite, mostrando un potencial robusto en generación de IP.


In [105]:
discussion = result.to_input_list()

In [106]:
discussion

[{'content': 'Explica por qué esta propuesta en innovadora, factible y alineada con la rúbrica.',
  'role': 'user'},
 {'arguments': '{}',
  'call_id': 'call_Wux8MoHg5zEbEgUcZCUsyrtP',
  'name': 'original_document',
  'type': 'function_call',
  'id': 'fc_6806cffe10708192bd84a7bcd313b50d046dbf87b97a46b6',
  'status': 'completed'},
 {'arguments': '{}',
  'call_id': 'call_vGpNNkjafFloFkMfkEAszP83',
  'name': 'rubric_format',
  'type': 'function_call',
  'id': 'fc_6806cffe2c288192a5b8b8c0e99f58dd046dbf87b97a46b6',
  'status': 'completed'},
 {'arguments': '{}',
  'call_id': 'call_FJRVxBZbrurOWMLIRkcVDXBW',
  'name': 'rubric_facts',
  'type': 'function_call',
  'id': 'fc_6806cffe3a748192a3f16eaf3e5a754f046dbf87b97a46b6',
  'status': 'completed'},
 {'call_id': 'call_Wux8MoHg5zEbEgUcZCUsyrtP',
  'output': '--- Page 1 ---\r\n \r\n1 \r\n\r\n \r\n\r\n\r\n--- Page 2 ---\r\n02 Tabla de Contenido \r\n \r\n \r\n\r\n01.-  \r\nTítulo  \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n  1 \r\n\r\n02.-  \r\nTab

## Independent Critique Round

In [ ]:

import asyncio

# Step 2: Independent Critique Round

msg = f"Perform your assigned task, I will pass you the msg of the proposal, and you will have to provide your critique. Be clear and concise. Provide an example to contextualize critique if possible. Be factual in your critique, a really good point. Don't raise concenrns out of nowhere or because you have to speak or yap. Make solid good points. Remember that we are funders, and we have to evaluate risks and opportunities. They a inventors trying to mature their techonology with us. But we also have to be objective. \n\n{discussion}"

result_1, result_2, result_3, result_4  = await asyncio.gather(
    Runner.run(
        devils_advocate_agent,
        input=msg,
        ),
    Runner.run(
        evidence_reviewer_agent,
        input=msg,
        ),
    Runner.run(
        feasibility_agent,
        input=msg,
        ),
    Runner.run(
        ethics_impact_agent,
        input=msg,
        )
)

In [120]:
from IPython.display import Markdown

result_1_markdown = str()
for observation in result_1.final_output.observations:
    result_1_markdown += f"\n- **Original:** \n\n{observation.original}\n- **Observation:** \n\n{observation.observation}\n"

# Display the content of result_1.final_output as Markdown
Markdown(result_1_markdown)


- **Original:** 

La propuesta tiene un mérito técnico elevado con una justificación científica y técnica bien definida. Está respaldada por publicaciones, patentes y una clara trayectoria académica del equipo, lo que fortalece su fundamento técnico.
- **Observation:** 

Aunque se destaca el mérito técnico de la propuesta, es relevante analizar si las patentes y publicaciones realmente cubren los aspectos innovadores propuestos o si solo refuerzan tecnologías preexistentes. Además, sería importante verificar si las aplicaciones mencionadas ya han demostrado tener un impacto significativo en contextos similares o si aún se enfrentan desafíos en la implementación real. Por ejemplo, si las patentes registradas son principalmente aplicaciones incrementales de tecnologías existentes, esto podría debilitar la percepción de innovación del proyecto. 

Analizar si realmente existe un uso novedoso de estas tecnologías es crucial para garantizar que la inversión no está replicando esfuerzos previos sin aportar avances sustanciales. Esto es especialmente relevante en el contexto competitivo de tecnologías de tratamiento de agua, donde siempre existe presión por innovaciones más efectivas.

- **Original:** 

La solución técnica es clara y coherente, combinando tecnología predictiva con tratamiento innovador de agua, lo que plantea una respuesta efectiva al problema descrito.
- **Observation:** 

Aunque la solución técnica es descrita como clara y coherente, es importante cuestionar cuán efectivo y probado es el uso de la tecnología predictiva en el contexto específico de Querétaro. La precisión de los modelos predictivos es crucial y su inexactitud podría comprometer parte de la solución integral propuesta. 

Por ejemplo, si los modelos de predicción no han sido adaptados adecuadamente a las condiciones meteorológicas locales, podrían no predecir con precisión los patrones de lluvia, afectando decisiones de implementación y efectividad general del sistema. Es vital asegurar que las soluciones predictivas no solo funcionen en teoría, sino que también se hayan validado en condiciones reales para justificar la confianza puesta en ellos.

- **Original:** 

El proyecto es claramente viable, con un enfoque modular adaptable a diferentes contextos y apoyado por un profundo análisis de riesgos y factibilidad.
- **Observation:** 

La viabilidad del proyecto es evidente según la presentación, pero hay ausencia de detalles sobre cómo se llevará a cabo el "profundo análisis de riesgos y factibilidad". Es importante entender los riesgos específicos identificados y las estrategias para mitigarlos, especialmente si apunta a adaptarse a diversos contextos. 

Por ejemplo, en el caso de aplicarse en áreas rurales sin acceso a electricidad, la dependencia de paneles solares podría no ser sostenible si el clima impide su funcionamiento óptimo. Requiere de análisis más detallado para asegurar que la tecnología es viable y sostenible a largo plazo en todos los contextos mencionados.


In [121]:
result_2_markdown = str()
result_2_markdown = str()
for observation in result_1.final_output.observations:
    result_2_markdown += f"\n- **Original:** \n\n{observation.original}\n- **Observation:** \n\n{observation.observation}\n"

# Display the content of result_1.final_output as Markdown
Markdown(result_2_markdown)


- **Original:** 

La propuesta tiene un mérito técnico elevado con una justificación científica y técnica bien definida. Está respaldada por publicaciones, patentes y una clara trayectoria académica del equipo, lo que fortalece su fundamento técnico.
- **Observation:** 

Aunque se destaca el mérito técnico de la propuesta, es relevante analizar si las patentes y publicaciones realmente cubren los aspectos innovadores propuestos o si solo refuerzan tecnologías preexistentes. Además, sería importante verificar si las aplicaciones mencionadas ya han demostrado tener un impacto significativo en contextos similares o si aún se enfrentan desafíos en la implementación real. Por ejemplo, si las patentes registradas son principalmente aplicaciones incrementales de tecnologías existentes, esto podría debilitar la percepción de innovación del proyecto. 

Analizar si realmente existe un uso novedoso de estas tecnologías es crucial para garantizar que la inversión no está replicando esfuerzos previos sin aportar avances sustanciales. Esto es especialmente relevante en el contexto competitivo de tecnologías de tratamiento de agua, donde siempre existe presión por innovaciones más efectivas.

- **Original:** 

La solución técnica es clara y coherente, combinando tecnología predictiva con tratamiento innovador de agua, lo que plantea una respuesta efectiva al problema descrito.
- **Observation:** 

Aunque la solución técnica es descrita como clara y coherente, es importante cuestionar cuán efectivo y probado es el uso de la tecnología predictiva en el contexto específico de Querétaro. La precisión de los modelos predictivos es crucial y su inexactitud podría comprometer parte de la solución integral propuesta. 

Por ejemplo, si los modelos de predicción no han sido adaptados adecuadamente a las condiciones meteorológicas locales, podrían no predecir con precisión los patrones de lluvia, afectando decisiones de implementación y efectividad general del sistema. Es vital asegurar que las soluciones predictivas no solo funcionen en teoría, sino que también se hayan validado en condiciones reales para justificar la confianza puesta en ellos.

- **Original:** 

El proyecto es claramente viable, con un enfoque modular adaptable a diferentes contextos y apoyado por un profundo análisis de riesgos y factibilidad.
- **Observation:** 

La viabilidad del proyecto es evidente según la presentación, pero hay ausencia de detalles sobre cómo se llevará a cabo el "profundo análisis de riesgos y factibilidad". Es importante entender los riesgos específicos identificados y las estrategias para mitigarlos, especialmente si apunta a adaptarse a diversos contextos. 

Por ejemplo, en el caso de aplicarse en áreas rurales sin acceso a electricidad, la dependencia de paneles solares podría no ser sostenible si el clima impide su funcionamiento óptimo. Requiere de análisis más detallado para asegurar que la tecnología es viable y sostenible a largo plazo en todos los contextos mencionados.


In [57]:
Markdown(result_3.final_output)

### Evaluación de la Propuesta

#### Factibilidad del Proyecto

1. **Mérito Técnico**: 
   - La propuesta tiene un fuerte respaldo técnico, basado en estudios previos y tecnologías desarrolladas en el campo de la oxidación electroquímica. La validación a nivel TRL 4 muestra un nivel significativo de desarrollo tecnológico y preparación para avanzar.

2. **Solución Técnica Propuesta**:
   - Se plantea una solución innovadora al integrar tecnologías de modelación predictiva y de tratamiento avanzado del agua, adaptadas a las necesidades específicas de Querétaro.

3. **Viabilidad de Implementación**:
   - El enfoque modular de la propuesta permite escalabilidad y adaptación a diversas circunstancias. Sin embargo, hay una falta de detalle en la disponibilidad específica de algunos recursos, lo que podría afectar la implementación.

4. **Equipo Proponente**:
   - La experiencia y liderazgo del equipo son claros, con habilidades complementarias que cubren todas las áreas necesarias para el proyecto.

5. **Justificación del Presupuesto**:
   - Aunque el presupuesto está bien alineado con los objetivos del proyecto, hay oportunidades de optimización y falta de detalles en algunas áreas.

6. **Impactos Esperados y Monitoreo**: 
   - Los impactos están claramente definidos, con un sólido plan de monitoreo y evaluación, asegurando que se puedan medir los beneficios sociales y ambientales.

7. **Plan de Trabajo**:
   - El plan es detallado, con hitos bien definidos y un cronograma realista, asegurando el cumplimiento de los objetivos dentro de los plazos establecidos.

8. **Propiedad Intelectual**:
   - Existen avances significativos en la generación de propiedad intelectual, con varias solicitudes de patentes en trámite.

#### Desafíos Potenciales

- **Disponibilidad de Recursos**: La falta de especificidad en algunos detalles de recursos podría representar un desafío en la práctica. Evaluar y asegurar estos recursos sería crucial para la implementación sin interrupciones.

- **Optimización Presupuestaria**: Mientras que el presupuesto está alineado con los objetivos, evaluaciones más detalladas podrían optimizar costos y mejorar la eficiencia en la asignación de recursos.

#### Conclusión

La propuesta es técnicamente sólida, innovadora y tiene un enfoque bien estructurado para la gestión hídrica en Querétaro a través de un esquema de economía circular. No obstante, se deben abordar algunas áreas de mejora, especialmente en la especificidad de los recursos y la optimización presupuestaria, para maximizar la eficacia y la eficiencia del proyecto.

In [58]:
Markdown(result_4.final_output)

### Evaluación Ética y de Impacto

La propuesta presentada para el aprovechamiento eficiente del agua de lluvia en Querétaro mediante tecnologías de captación y tratamiento representa una innovación importante, sin embargo, hay varios aspectos éticos y de impacto que deben ser cuidadosamente considerados:

1. **Impacto Ambiental:**
   - **Beneficios**: La propuesta podría reducir la extracción de aguas subterráneas y superficiales, contribuyendo a la sostenibilidad del recurso hídrico.
   - **Riesgos**: La instalación de dispositivos en áreas naturales podría alterar ecosistemas locales. Además, el tratamiento de aguas grises y pluviales debe garantizar que no introducirá contaminantes residuales al medio ambiente.

2. **Equidad y Acceso:**
   - **Equidad de acceso**: Es crucial que el sistema desarrollado no se limite sólo a comunidades con cierto nivel económico o infraestructura preexistente. La implementación debe ser inclusiva, beneficiando también a zonas rurales y marginadas.
   - **Justicia Social**: El despliegue de esta tecnología debe considerar la equidad en el acceso a los beneficios derivados del proyecto.

3. **Privacidad y Datos:**
   - **Protección de Datos**: Con el uso de modelos predictivos y posiblemente el seguimiento del uso de agua, deben establecerse protocolos claros para la gestión de datos personales y la privacidad de las comunidades.

4. **Transparencia y Consenso:**
   - **Participación Comunitaria**: El éxito y la aceptación del proyecto dependen de la participación de las comunidades afectadas. Es fundamental el consenso informado y la incorporación de las perspectivas locales en el diseño e implementación del proyecto.
   - **Comunicación**: La claridad y transparencia en la comunicación sobre los beneficios, riesgos y cambios que conlleva el proyecto son cruciales para ganar la confianza pública.

5. **Seguridad y Mantenimiento:**
   - **Seguridad Operativa**: Los sistemas, especialmente los de tratamiento electroquímico, deben cumplir con estrictas normas de seguridad durante su instalación, operación y mantenimiento.
   - **Mantenimiento Responsable**: La propuesta debe incluir un plan de mantenimiento robusto que asegure la sostenibilidad a largo plazo del sistema sin depender excesivamente de recursos tecnológicos externos.

6. **Impacto a Largo Plazo:**
   - **Sostenibilidad**: Aunque el proyecto tiene potencial para el ahorro de agua, los beneficios deben ser medidos bajo un marco de responsabilidad ambiental a largo plazo.
   - **Reevaluación Continua**: Establecer un marco para reevaluar el proyecto a medida que se desarrolla para adaptar y mitigar cualquier impacto negativo emergente.

### Recomendaciones:

- **Incorporar Evaluaciones Éticas Formales:** Se debe realizar una evaluación ética formal que aborde los potenciales dilemas y riesgos señalados.
- **Fomentar la Inclusión y la Justicia:** Asegurar que todas las comunidades, independientemente de su situación económica, se beneficien equitativamente del proyecto.
- **Desarrollar un Protocolo Clásico de Gestión de Datos:** Se debe crear un protocolo claro de gestión de datos para abordar temas de privacidad.
- **Mejorar la Participación Comunitaria:** Aumentar el involucramiento comunitario a través de talleres y reuniones de consenso.
- **Plan de Mantenimiento Proactivo:** Desarrollar un plan de mantenimiento proactivo que involucre capacitación local para sentar bases sólidas en el manejo del sistema.

En resumen, el proyecto presenta una oportunidad significativa para mejorar la gestión hídrica, pero debe ser abordado con responsabilidad ética y sensibilidad hacia los impactos sociales y ambientales.

In [132]:
discussion.append(
    {"role": "assistant", 
     "content": [
        {"agent": "Reviewer_1", "content": result_1.final_output.observations},
        {"agent": "Reviewer_2", "content": result_2.final_output.observations},
        {"agent": "Reviewer_3", "content": result_3.final_output.observations},
        {"agent": "Reviewer_4", "content": result_4.final_output.observations},
         ]}
)

## Cross-critique Round

In [175]:
# Step 3: Cross-Critique Round

msg = f"""
You previously examined a proposal and provided feedback. Now, you will cross-examine the comments of other agents regarding the initially provided document. Your task is to critically evaluate their observations **only if you feel strongly against the observation**. If you do not have a strong opinion, you are not obligated to respond.

Focus on:
1. Identifying any gaps, inconsistencies, or unsupported claims in their feedback if you strongly disagree.
2. Providing constructive and specific reasoning to support your stance.
3. Remember that we evaluate so our company can catch good proposals to mature. But also having in mind that we can't support ill-founded ones.

The discussion so far is provided below for your review:

{discussion}
"""

devils_advocate_agent.output_type = Observations_critiques
evidence_reviewer_agent.output_type = Observations_critiques 
feasibility_agent.output_type = Observations_critiques
ethics_impact_agent.output_type = Observations_critiques

result_1, result_2, result_3, result_4  = await asyncio.gather(
    Runner.run(
        devils_advocate_agent,
        input=msg + "\nNOTE: You are reviewer 1 in the evidences. Don't evaluate yourself.",
        ),
    Runner.run(
        evidence_reviewer_agent,
        input=msg + "\nNOTE: You are reviewer 2 in the evidences. Don't evaluate yourself.",
        ),
    Runner.run(
        feasibility_agent,
        input=msg + "\nNOTE: You are reviewer 3 in the evidences. Don't evaluate yourself.",
        ),
    Runner.run(
        ethics_impact_agent,
        input=msg + "\nNOTE: You are reviewer 4 in the evidences. Don't evaluate yourself.",
        )
)

In [176]:
def pretty_print(data, indent=0):
    """
    Recursively prints nested dictionaries, lists, and strings in a readable format.
    
    Args:
        data: The data structure to print (dict, list, str, etc.).
        indent: The current indentation level (used for nested structures).
    """
    # Define the indentation string
    spacing = "    " * indent

    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{Grn}{spacing}{key}{Rst}:")
            pretty_print(value, indent + 1)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print(f"{Mag}{spacing}- [{index}]{Rst}")
            pretty_print(item, indent + 1)
    else:
        # Print strings or other primitive types
        print(f"{spacing}{data}")

In [177]:
pretty_print(result_1.final_output.model_dump())
pretty_print(result_2.final_output.model_dump())
pretty_print(result_3.final_output.model_dump())
pretty_print(result_4.final_output.model_dump())

observations:
    - [0]
        reviewer:
            Reviewer_2
        original:
            El proyecto es claramente viable, con un enfoque modular adaptable a diferentes contextos y apoyado por un profundo análisis de riesgos y factibilidad.
        observation:
            Estoy en desacuerdo en que el análisis de riesgos y factibilidad esté claramente expuesto. La propuesta menciona la viabilidad y modularidad como fortalezas, pero no proporciona suficientes detalles específicos sobre cómo se llevará a cabo el análisis de riesgos. Esto es crucial para respaldar la afirmación de viabilidad. En proyectos complejos, es necesario que las estrategias de mitigación de riesgos sean detalladas, especialmente cuando se implementan tecnologías en áreas rurales sin acceso a electricidad convencional. Más detalles sobre estos aspectos serían necesarios para considerar la propuesta completamente viable.
    - [1]
        reviewer:
            Reviewer_3
        original:
            El proye

In [178]:
result_1.final_output

Observations_critiques(observations=[Observation_to_critiques(reviewer='Reviewer_2', original='El proyecto es claramente viable, con un enfoque modular adaptable a diferentes contextos y apoyado por un profundo análisis de riesgos y factibilidad.', observation='Estoy en desacuerdo en que el análisis de riesgos y factibilidad esté claramente expuesto. La propuesta menciona la viabilidad y modularidad como fortalezas, pero no proporciona suficientes detalles específicos sobre cómo se llevará a cabo el análisis de riesgos. Esto es crucial para respaldar la afirmación de viabilidad. En proyectos complejos, es necesario que las estrategias de mitigación de riesgos sean detalladas, especialmente cuando se implementan tecnologías en áreas rurales sin acceso a electricidad convencional. Más detalles sobre estos aspectos serían necesarios para considerar la propuesta completamente viable.'), Observation_to_critiques(reviewer='Reviewer_3', original='El proyecto es claramente viable, con un enfoq

In [179]:
result_1_markdown = str()
for observation in result_1.final_output.Observations_critiques:
    result_1_markdown += f"\n- **Original:** \n\n{observation.original}\n- **Observation:** \n\n{observation.observation}\n"

# Display the content of result_1.final_output as Markdown
Markdown(result_1_markdown)

AttributeError: 'Observations_critiques' object has no attribute 'Observations_critiques'